In [1]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras import layers, models
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications import ResNet101
from keras.applications import DenseNet201, DenseNet169, DenseNet121
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import Xception
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support, cohen_kappa_score
import pandas as pd

init_notebook_mode(connected=True)
RANDOM_SEED = 123

Using TensorFlow backend.


# Data Import and Preprocessing

In [1]:
IMG_PATH = 'brain_tumor_dataset/'
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/' + CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/' + CLASS.upper() + '/' + FILE_NAME) 

NameError: name 'os' is not defined

In [2]:
def load_data(dir_path, img_size=(100, 100)):
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print('{} images loaded from {} directory.'.format(len(X), dir_path))
    return X, y, labels
    

In [4]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224, 224)

X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.28it/s]
C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

242 images loaded from TRAIN/ directory.
19 images loaded from TEST/ directory.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11.33it/s]

92 images loaded from VAL/ directory.


In [5]:
def preprocess_imgs(set_name, img_size):
    
    # Resize and apply VGG-15 preprocessing
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [6]:
X_train_prep = preprocess_imgs(set_name=X_train, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val, img_size=IMG_SIZE)

# CNN Pre-trained Models

## Data Augmentation

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='binary',
    seed=RANDOM_SEED
)


Found 242 images belonging to 2 classes.
Found 92 images belonging to 2 classes.


# Pre-trained Networks Evaluation

In [8]:
nets_evaluation_df = pd.DataFrame(columns=['Training Accuracy', 'Validation Accuracy', 'Test Accuracy', 'F1 Measure', 'Kappa', 'ROC Area'])
EPOCHS = 30
NUM_CLASSES = 1
trained_nets = {VGG16 : 'VGG16', VGG19 : 'VGG19',  ResNet50 : 'ResNet50', ResNet101 : 'ResNet101', DenseNet201 : 'DenseNet201',
               DenseNet169 : 'DenseNet169', DenseNet121 : 'DenseNet121', InceptionV3 : 'InceptionV3', InceptionResNetV2 : 'InceptionResNetV2',
               Xception : 'Xception', MobileNetV2 : 'MobileNetV2', MobileNet : 'MobileNet'}

In [9]:

for net in trained_nets.keys():
    # loading base model
    base_model = net(
    weights='imagenet',
    include_top=False,
    input_shape=IMG_SIZE + (3,))
    
    

    model= Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))

    model.layers[0].trainable = False

    model.compile(
        loss='binary_crossentropy',
        optimizer=RMSprop(lr=1e-4),
        metrics=['accuracy'])

    #model.summary()
    
    
    history = model.fit(
                    train_generator, steps_per_epoch=len(train_generator),
                    epochs=EPOCHS, validation_data=validation_generator,
                    validation_steps=len(validation_generator))
    
    # validate on test set    
    predictions = model.predict(X_test_prep)
    predictions = [1 if x>0.5 else 0 for x in predictions]

    acc = history.history['accuracy']
    
    val_acc = history.history['val_accuracy']

    test_accuracy = accuracy_score(y_test, predictions)

    kappa_score = cohen_kappa_score(y_test, predictions)

    roc_area = roc_auc_score(y_test, predictions)

    (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, predictions, average='macro')
    
    measures = [acc[-1], val_acc[-1], test_accuracy, fscore, kappa_score, roc_area]

    nets_evaluation_df.loc[trained_nets[net]] = measures

Epoch 1/30
16/16 [==============================] - 9s 549ms/step - loss: 5.4448 - accuracy: 0.5000 - val_loss: 5.5377 - val_accuracy: 0.5217
Epoch 2/30
16/16 [==============================] - 2s 141ms/step - loss: 4.9313 - accuracy: 0.5868 - val_loss: 2.0799 - val_accuracy: 0.6630
Epoch 3/30
16/16 [==============================] - 2s 149ms/step - loss: 4.3335 - accuracy: 0.6570 - val_loss: 2.6880 - val_accuracy: 0.6630
Epoch 4/30
16/16 [==============================] - 2s 148ms/step - loss: 2.7555 - accuracy: 0.6653 - val_loss: 0.6554 - val_accuracy: 0.7391
Epoch 5/30
16/16 [==============================] - 2s 147ms/step - loss: 3.4627 - accuracy: 0.6446 - val_loss: 1.6168 - val_accuracy: 0.7826
Epoch 6/30
16/16 [==============================] - 3s 162ms/step - loss: 2.2604 - accuracy: 0.7190 - val_loss: 1.2271 - val_accuracy: 0.8152
Epoch 7/30
16/16 [==============================] - 3s 175ms/step - loss: 3.9661 - accuracy: 0.7521 - val_loss: 0.4778 - val_accuracy: 0.8370
Epoch 

C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



Epoch 1/30
16/16 [==============================] - 8s 478ms/step - loss: 0.8878 - accuracy: 0.6777 - val_loss: 0.6378 - val_accuracy: 0.6630
Epoch 2/30
16/16 [==============================] - 2s 144ms/step - loss: 0.5722 - accuracy: 0.7851 - val_loss: 0.3904 - val_accuracy: 0.6196
Epoch 3/30
16/16 [==============================] - 3s 163ms/step - loss: 0.6194 - accuracy: 0.7769 - val_loss: 0.7916 - val_accuracy: 0.5870
Epoch 4/30
16/16 [==============================] - 3s 168ms/step - loss: 0.5508 - accuracy: 0.7810 - val_loss: 1.3082 - val_accuracy: 0.5326
Epoch 5/30
16/16 [==============================] - 3s 162ms/step - loss: 0.5551 - accuracy: 0.8347 - val_loss: 1.4773 - val_accuracy: 0.6413
Epoch 6/30
16/16 [==============================] - 3s 158ms/step - loss: 0.6172 - accuracy: 0.8223 - val_loss: 0.5901 - val_accuracy: 0.7283
Epoch 7/30
16/16 [==============================] - 3s 163ms/step - loss: 0.5081 - accuracy: 0.8471 - val_loss: 0.4759 - val_accuracy: 0.7500
Epoch 

C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch 1/30
16/16 [==============================] - 12s 765ms/step - loss: 0.8985 - accuracy: 0.6240 - val_loss: 4.6937 - val_accuracy: 0.6196
Epoch 2/30
16/16 [==============================] - 3s 196ms/step - loss: 0.6018 - accuracy: 0.7479 - val_loss: 6.5361 - val_accuracy: 0.6196
Epoch 3/30
16/16 [==============================] - 3s 196ms/step - loss: 0.5627 - accuracy: 0.7727 - val_loss: 7.3452 - val_accuracy: 0.6304
Epoch 4/30
16/16 [==============================] - 3s 196ms/step - loss: 0.5679 - accuracy: 0.7727 - val_loss: 2.7874 - val_accuracy: 0.5870
Epoch 5/30
16/16 [==============================] - 3s 196ms/step - loss: 0.5547 - accuracy: 0.8099 - val_loss: 1.7292 - val_accuracy: 0.6739
Epoch 6/30
16/16 [==============================] - 3s 197ms/step - loss: 0.4304 - accuracy: 0.7893 - val_loss: 2.5741 - val_accuracy: 0.5109
Epoch 7/30
16/16 [==============================] - 3s 196ms/step - loss: 0.3797 - accuracy: 0.8264 - val_loss: 1.7919 - val_accuracy: 0.5435
Epoch

...

Epoch 1/30
16/16 [==============================] - 10s 643ms/step - loss: 0.8087 - accuracy: 0.5826 - val_loss: 0.2192 - val_accuracy: 0.6630
Epoch 2/30
16/16 [==============================] - 2s 96ms/step - loss: 0.7800 - accuracy: 0.6570 - val_loss: 1.6192 - val_accuracy: 0.7065
Epoch 3/30
16/16 [==============================] - 3s 191ms/step - loss: 0.5921 - accuracy: 0.7190 - val_loss: 2.9186 - val_accuracy: 0.6739
Epoch 4/30
16/16 [==============================] - 3s 191ms/step - loss: 0.5824 - accuracy: 0.7149 - val_loss: 8.3897 - val_accuracy: 0.4457
Epoch 5/30
16/16 [==============================] - 3s 211ms/step - loss: 0.5795 - accuracy: 0.7314 - val_loss: 6.3314 - val_accuracy: 0.5435
Epoch 6/30
16/16 [==============================] - 3s 202ms/step - loss: 0.5898 - accuracy: 0.7397 - val_loss: 4.7859 - val_accuracy: 0.6196
Epoch 7/30
16/16 [==============================] - 3s 206ms/step - loss: 0.6002 - accuracy: 0.7273 - val_loss: 3.2243 - val_accuracy: 0.6413
Epoch 

C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch 1/30
16/16 [==============================] - 7s 442ms/step - loss: 0.6722 - accuracy: 0.6322 - val_loss: 6.8903 - val_accuracy: 0.6196
Epoch 2/30
16/16 [==============================] - 3s 169ms/step - loss: 0.5338 - accuracy: 0.7438 - val_loss: 10.4718 - val_accuracy: 0.6196
Epoch 3/30
16/16 [==============================] - 3s 185ms/step - loss: 0.4302 - accuracy: 0.8017 - val_loss: 12.9370 - val_accuracy: 0.6196
Epoch 4/30
16/16 [==============================] - 3s 185ms/step - loss: 0.4617 - accuracy: 0.7769 - val_loss: 5.9569 - val_accuracy: 0.6196
Epoch 5/30
16/16 [==============================] - 3s 184ms/step - loss: 0.4219 - accuracy: 0.7934 - val_loss: 3.3038 - val_accuracy: 0.6196
Epoch 6/30
16/16 [==============================] - 3s 185ms/step - loss: 0.3677 - accuracy: 0.8430 - val_loss: 3.1218 - val_accuracy: 0.6196
Epoch 7/30
16/16 [==============================] - 3s 183ms/step - loss: 0.4315 - accuracy: 0.8099 - val_loss: 9.5497 - val_accuracy: 0.6196
Epoc

C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch 1/30
16/16 [==============================] - 5s 328ms/step - loss: 1.0016 - accuracy: 0.6074 - val_loss: 1.0736 - val_accuracy: 0.4565
Epoch 2/30
16/16 [==============================] - 1s 85ms/step - loss: 0.8390 - accuracy: 0.7107 - val_loss: 1.0208 - val_accuracy: 0.4891
Epoch 3/30
16/16 [==============================] - 3s 180ms/step - loss: 0.7225 - accuracy: 0.7190 - val_loss: 1.5399 - val_accuracy: 0.4348
Epoch 4/30
16/16 [==============================] - 3s 176ms/step - loss: 0.5960 - accuracy: 0.7355 - val_loss: 2.0200 - val_accuracy: 0.3913
Epoch 5/30
16/16 [==============================] - 3s 181ms/step - loss: 0.6315 - accuracy: 0.7645 - val_loss: 2.0040 - val_accuracy: 0.4022
Epoch 6/30
16/16 [==============================] - 3s 174ms/step - loss: 0.3945 - accuracy: 0.8223 - val_loss: 1.1260 - val_accuracy: 0.4457
Epoch 7/30
16/16 [==============================] - 3s 176ms/step - loss: 0.5510 - accuracy: 0.7893 - val_loss: 1.4082 - val_accuracy: 0.5000
Epoch 8

In [10]:
nets_evaluation_df

,Training Accuracy,Validation Accuracy,Test Accuracy,F1 Measure,Kappa,ROC Area
VGG16,0.855372,0.967391,0.947368,0.946779,0.893855,0.944444
VGG19,0.863636,0.956522,0.947368,0.946779,0.893855,0.944444
ResNet50,0.933884,0.760870,0.684211,0.634615,0.344828,0.666667
ResNet101,0.946281,0.793478,0.789474,0.773810,0.568182,0.777778
DenseNet201,0.925620,0.619565,0.473684,0.321429,0.000000,0.500000
DenseNet169,0.946281,0.728261,0.368421,0.366667,-0.266667,0.366667
DenseNet121,0.876033,0.619565,0.473684,0.321429,0.000000,0.500000
InceptionV3,0.834711,0.554348,0.421053,0.421053,-0.154696,0.422222
InceptionResNetV2,0.900826,0.619565,0.473684,0.321429,0.000000,0.500000
Xception,0.921488,0.619565,0.473684,0.321429,0.000000,0.500000


## Custom CNN Models

In [14]:
input_shape = (224, 224, 3) 
NUM_CLASSES = 1

In [15]:

def model_cnn2():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn2 = model_cnn2()



model_cnn2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)     

In [58]:
def model_cnn3():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn3 = model_cnn3()

model_cnn3.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_308 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_402 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_308 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_309 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_403 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_309 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 112, 112, 32)    

In [64]:
def model_cnn4():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn4 = model_cnn4()

model_cnn4.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_311 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_405 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_311 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_312 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_406 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_312 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 112, 112, 32)    

In [65]:
def model_cnn5():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn5 = model_cnn5()

model_cnn5.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_315 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_409 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_315 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_316 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_410 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_316 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 112, 112, 32)    

In [66]:
def model_cnn6():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn6 = model_cnn6()

model_cnn6.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_320 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_414 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_320 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_321 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_415 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_321 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 112, 112, 32)    

In [67]:
def model_cnn7():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn7 = model_cnn7()

model_cnn7.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_326 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_420 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_326 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_327 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_421 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_327 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 112, 112, 32)    

In [68]:
def model_cnn8():
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))

    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
        
    return model

model_cnn8 = model_cnn8()

model_cnn8.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_333 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
activation_427 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_333 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_334 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_428 (Activation)  (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_334 (Bat (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 112, 112, 32)    

### Custom Models Evaluations Table

In [70]:
custom_models = {model_cnn2 : '2 X CNN', model_cnn3 : '3 X CNN', model_cnn4 : '4 X CNN', model_cnn5 : '5 X CNN',
                model_cnn6 : '6 X CNN', model_cnn7 : '7 X CNN', model_cnn8 : '8 X CNN'}
custom_cnn_evaluation_df = pd.DataFrame(columns=['Training Accuracy', 'Validation Accuracy', 'Test Accuracy', 'F1 Measure', 'Kappa', 'ROC Area'])

In [72]:
for model in custom_models.keys():
    
    model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy'])
    
    history = model.fit(
    train_generator, 
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))
    
    # validate on test set
    predictions = model.predict(X_test_prep)
    predictions = [1 if x>0.5 else 0 for x in predictions]

    acc = history.history['accuracy']
    
    val_acc = history.history['val_accuracy']

    test_accuracy = accuracy_score(y_test, predictions)

    kappa_score = cohen_kappa_score(y_test, predictions)

    roc_area = roc_auc_score(y_test, predictions)

    (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, predictions, average='macro')
    
    measures = [acc[-1], val_acc[-1], test_accuracy, fscore, kappa_score, roc_area]

    custom_cnn_evaluation_df.loc[custom_models[model]] = measures

Epoch 1/30
16/16 [==============================] - 4s 240ms/step - loss: 1.6422 - accuracy: 0.7231 - val_loss: 0.8556 - val_accuracy: 0.8043
Epoch 2/30
16/16 [==============================] - 4s 221ms/step - loss: 1.4314 - accuracy: 0.7686 - val_loss: 0.0131 - val_accuracy: 0.7717
Epoch 3/30
16/16 [==============================] - 3s 216ms/step - loss: 1.5893 - accuracy: 0.7025 - val_loss: 1.9828 - val_accuracy: 0.8261
Epoch 4/30
16/16 [==============================] - 3s 213ms/step - loss: 1.5678 - accuracy: 0.7562 - val_loss: 1.5120 - val_accuracy: 0.7935
Epoch 5/30
16/16 [==============================] - 3s 212ms/step - loss: 1.4784 - accuracy: 0.7603 - val_loss: 0.8222 - val_accuracy: 0.7935
Epoch 6/30
16/16 [==============================] - 3s 217ms/step - loss: 1.3419 - accuracy: 0.7438 - val_loss: 4.2235 - val_accuracy: 0.6087
Epoch 7/30
16/16 [==============================] - 4s 241ms/step - loss: 1.6547 - accuracy: 0.6983 - val_loss: 0.5335 - val_accuracy: 0.8043
Epoch 

16/16 [==============================] - 3s 213ms/step - loss: 0.8740 - accuracy: 0.7438 - val_loss: 0.2895 - val_accuracy: 0.8478
Epoch 26/30
16/16 [==============================] - 3s 207ms/step - loss: 0.8806 - accuracy: 0.7066 - val_loss: 0.2064 - val_accuracy: 0.8370
Epoch 27/30
16/16 [==============================] - 4s 222ms/step - loss: 1.0041 - accuracy: 0.7273 - val_loss: 0.1701 - val_accuracy: 0.8478
Epoch 28/30
16/16 [==============================] - 3s 214ms/step - loss: 0.8705 - accuracy: 0.7190 - val_loss: 0.4621 - val_accuracy: 0.7935
Epoch 29/30
16/16 [==============================] - 3s 213ms/step - loss: 0.8593 - accuracy: 0.7025 - val_loss: 0.2490 - val_accuracy: 0.7826
Epoch 30/30
16/16 [==============================] - 3s 211ms/step - loss: 0.9387 - accuracy: 0.7479 - val_loss: 0.4875 - val_accuracy: 0.8478
Epoch 1/30
16/16 [==============================] - 5s 293ms/step - loss: 1.1880 - accuracy: 0.5785 - val_loss: 0.6602 - val_accuracy: 0.7065
Epoch 2/30
1

In [73]:
custom_cnn_evaluation_df

,Training Accuracy,Validation Accuracy,Test Accuracy,F1 Measure,Kappa,ROC Area
2 X CNN,0.793388,0.836957,0.842105,0.842105,0.685083,0.844444
3 X CNN,0.785124,0.836957,0.789474,0.788889,0.582418,0.794444
4 X CNN,0.752066,0.826087,0.789474,0.784091,0.586957,0.800000
5 X CNN,0.747934,0.847826,0.789474,0.788889,0.582418,0.794444
6 X CNN,0.723140,0.793478,0.789474,0.788889,0.577778,0.788889
7 X CNN,0.698347,0.739130,0.789474,0.784091,0.586957,0.800000
8 X CNN,0.739669,0.706522,0.684211,0.660714,0.387097,0.700000


In [74]:
nets_evaluation_df = pd.concat([custom_cnn_evaluation_df, nets_evaluation_df])

In [76]:
nets_evaluation_df.to_excel("nets_evaluation.xlsx")

In [77]:
nets_evaluation_df

,Training Accuracy,Validation Accuracy,Test Accuracy,F1 Measure,Kappa,ROC Area
2 X CNN,0.793388,0.836957,0.842105,0.842105,0.685083,0.844444
3 X CNN,0.785124,0.836957,0.789474,0.788889,0.582418,0.794444
4 X CNN,0.752066,0.826087,0.789474,0.784091,0.586957,0.800000
5 X CNN,0.747934,0.847826,0.789474,0.788889,0.582418,0.794444
6 X CNN,0.723140,0.793478,0.789474,0.788889,0.577778,0.788889
7 X CNN,0.698347,0.739130,0.789474,0.784091,0.586957,0.800000
8 X CNN,0.739669,0.706522,0.684211,0.660714,0.387097,0.700000
VGG16,0.855372,0.945652,0.894737,0.894444,0.788889,0.894444
VGG19,0.859504,0.934783,0.789474,0.788889,0.577778,0.788889
ResNet50,0.954545,0.771739,0.684211,0.634615,0.344828,0.666667
